# LangChain Workshop: Building an AI-Powered Chatbot
### CCDS Tech for Good 2026 Hackathon

**Workshop Goals:**
- Understand LangChain fundamentals
- Build conversational AI agents
- Create production-ready chatbot classes
- Master prompt engineering techniques

**What you'll build:**
- Basic chatbot with memory
- AI companion with personality
- Tool-using agents
- Reusable chatbot framework

# 1. LangChain Fundamentals

## 1.1 Your First LLM Call

In [2]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Initialize the LLM
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    deployment_name="gpt-5-nano",
)

# Create messages
messages = [
    SystemMessage(content="You are a helpful AI assistant."),
    HumanMessage(content="Explain what LangChain is in one sentence.")
]

# Get response
response = llm.invoke(messages)
print(response.content)

LangChain is a framework for building AI applications with large language models,提供 modular components like prompts, chains, and agents, plus easy integrations with tools and data sources.


###  Understanding the Components

**1. SystemMessage**: Defines the AI's role and behavior
- Sets personality and expertise
- Provides context and constraints
- Remains consistent across conversation

**2. HumanMessage**: User's input to the AI
- The query or prompt
- Changes with each interaction

**3. Temperature**: Controls randomness (0-1)
- `0.0` → Deterministic, focused
- `0.7` → Balanced (recommended)
- `1.0` → Creative, varied

## 2.2 Adding Conversation Memory

In [7]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# 1. Initialize the Azure LLM
# Ensure 'langchain-openai' is installed via: uv add langchain-openai
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    deployment_name="gpt-5-nano"
)

# 2. Setup the Prompt with a 'history' placeholder
# This replaces the internal 'ConversationChain' template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

# 3. Create the Chain (LCEL pipe syntax)
chain = prompt | llm

# 4. Define Memory Storage
# Using ChatMessageHistory instead of the legacy ConversationBufferMemory
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 5. Wrap the chain with History logic
# This is the modern replacement for ConversationChain
conversation_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

# 6. Execution
config = {"configurable": {"session_id": "robot_dev_test"}}

print("Turn 1:")
res1 = conversation_with_history.invoke({"input": "Hi! My name is Sarah."}, config)
print(f"AI: {res1.content}\n")

print("Turn 2:")
res2 = conversation_with_history.invoke({"input": "What's my name?"}, config)
print(f"AI: {res2.content}\n")

print("Turn 3:")
res3 = conversation_with_history.invoke({"input": "What did we just talk about?"}, config)
print(f"AI: {res3.content}")

Turn 1:
AI: Hi Sarah! Nice to meet you. How can I help today? I can answer questions, help with writing, brainstorm ideas, plan projects, practice a skill, or just chat. Tell me a bit about what you’re interested in or what you’d like to work on, and we’ll take it from there.

Turn 2:
AI: Your name is Sarah. Nice to meet you again, Sarah—would you like me to call you by a nickname or just Sarah?

Turn 3:
AI: We talked about your name. You said you’re Sarah, I confirmed that and asked whether you’d like me to call you Sarah or use a nickname. Would you prefer a nickname, or should I just stick with Sarah?


## 2.3 Different Memory Types

In [9]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_community.chat_message_histories import ChatMessageHistory

# Initialize LLM for Summary Memory
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    deployment_name="gpt-5-nano"
)

# --- 1. Buffer Memory (Stores Everything) ---
history = ChatMessageHistory()
history.add_user_message("Hi!")
history.add_ai_message("Hello!")
print("=== Buffer Memory ===")
print(history.messages)

print("\n" + "="*50 + "\n")

# --- 2. Window Memory (Last K interactions) ---
# In modern LangChain, you simply slice the message list.
# This is much more 'Pythonic' and efficient for high-performance C++ integration.
all_messages = [
    HumanMessage(content="Message 1"), AIMessage(content="Response 1"),
    HumanMessage(content="Message 2"), AIMessage(content="Response 2"),
    HumanMessage(content="Message 3"), AIMessage(content="Response 3"),
]
k = 2
window_messages = all_messages[-(k*2):] # Multiply by 2 because 1 interaction = User + AI
print(f"=== Window Memory (k={k}) ===")
for msg in window_messages:
    print(f"{type(msg).__name__}: {msg.content}")

print("\n" + "="*50 + "\n")

# --- 3. Summary Memory (LLM-based Summarization) ---
# Instead of a hidden class, you explicitly ask the LLM to summarize.
def summarize_messages(existing_summary, new_messages):
    prompt = f"Current summary: {existing_summary}\n\nNew messages: {new_messages}\n\nUpdate the summary."
    return llm.invoke(prompt).content

current_summary = "The user is asking about ML."
new_talk = "We discussed how it applies to Underwater Unmanned Vehicles (UUVs)."
updated_summary = summarize_messages(current_summary, new_talk)

print("=== Summary Memory ===")
print(f"Updated Summary: {updated_summary}")

=== Buffer Memory ===
[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello!', additional_kwargs={}, response_metadata={})]


=== Window Memory (k=2) ===
HumanMessage: Message 2
AIMessage: Response 2
HumanMessage: Message 3
AIMessage: Response 3


=== Summary Memory ===
Updated Summary: Updated summary: We started with a general ML question and then discussed applying machine learning to Underwater Unmanned Vehicles (UUVs), including how ML can aid navigation, perception, control, autonomy, and mission planning in underwater environments, along with challenges (limited bandwidth, acoustic comms, sensing noise) and approaches (supervised/unsupervised learning, reinforcement learning, onboard edge processing).


## 2.4 Prompt Templates

In [11]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import PromptTemplate

# 1. Initialize LLM (Ensure this is already set up)
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    deployment_name="gpt-5-nano"
)

# 2. Create the Prompt Template
template = """
You are a {personality} AI assistant specialized in {domain}.

User's question: {question}

Provide a {tone} response.
"""

prompt = PromptTemplate.from_template(template)

# 3. Create the Chain (Modern LCEL Pipe Syntax)
# This replaces: chain = LLMChain(llm=llm, prompt=prompt)
chain = prompt | llm

# 4. Execution
# We use .invoke() instead of .predict()

print("=== Friendly Tutor ===")
response1 = chain.invoke({
    "personality": "friendly and encouraging",
    "domain": "mathematics",
    "question": "What is the Pythagorean theorem?",
    "tone": "simple and easy to understand"
})
# Note: .invoke() returns a message object, so we access .content
print(response1.content)

print("\n" + "="*60 + "\n")

print("=== Professional Advisor ===")
response2 = chain.invoke({
    "personality": "professional and analytical",
    "domain": "career development",
    "question": "How do I prepare for a data science interview?",
    "tone": "structured and actionable"
})
print(response2.content)

=== Friendly Tutor ===
The Pythagorean theorem applies to right triangles. It says:

- If a and b are the two short sides (the legs) and c is the longest side (the hypotenuse), then a^2 + b^2 = c^2.

Simple example:
- For a = 3 and b = 4, c^2 = 3^2 + 4^2 = 9 + 16 = 25, so c = 5.

Ways to use it:
- If you know the two legs, you can find the hypotenuse: c = sqrt(a^2 + b^2).
- If you know one leg and the hypotenuse, you can find the other leg: b = sqrt(c^2 - a^2).

Note: It only works for right triangles (one angle of 90 degrees).


=== Professional Advisor ===
Here's a structured, actionable playbook to prepare for a data science interview. It covers what to study, how to practice, and how to present yourself across common formats and company types.

1) Know your target and tailor your approach
- Roles to distinguish:
  - Data Scientist / Applied Scientist: strong modeling, experimentation, business impact, some production thinking.
  - ML Engineer: model deployment, scalability, pipelin

# 3. Building Your AI Companion

## 3.1 Basic Chatbot with Personality

In [ ]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# 1. Define chatbot personality
# We use a 'System' message to define Alex's persona
chatbot_personality = """
You are Alex, a friendly and knowledgeable AI companion. You are:
- Patient and understanding
- Enthusiastic about learning and helping
- Conversational and warm
- Clear and concise in explanations
Always respond in a friendly, natural way. Keep responses focused and helpful.
"""

# 2. Create the modern Chat Prompt Template
# This replaces the old 'PromptTemplate' and handles memory automatically
prompt = ChatPromptTemplate.from_messages([
    ("system", chatbot_personality),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

# 3. Initialize Azure LLM
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    deployment_name="gpt-35-turbo", 
    temperature=0.7
)

# 4. Memory Management (The Window Memory Logic)
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        # ChatMessageHistory stores all messages for the session
        store[session_id] = ChatMessageHistory()
    
    # Implementing 'Window' logic (k=5)
    # This keeps the context window clean for high-performance inference
    if len(store[session_id].messages) > 10: # k=5 interactions * 2 messages each
        store[session_id].messages = store[session_id].messages[-10:]
        
    return store[session_id]

# 5. Build the modern chain
chain = prompt | llm

# Wrap with history logic
conversation = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

# 6. Test the Chatbot
config = {"configurable": {"session_id": "alex_test_1"}}

def ask_alex(user_input):
    print(f"You: {user_input}")
    response = conversation.invoke({"input": user_input}, config)
    print(f"Alex: {response.content}\n")

print("Testing Alex the AI Companion:\n")
ask_alex("Hi Alex! How are you today?")
ask_alex("Can you help me with my studies?")
ask_alex("What was the first thing I asked you?")

Testing Alex the AI Companion:

You: Hi Alex! How are you today?


NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}

## 3.2 Reusable Chatbot Class

In [15]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

class AICompanion:
    def __init__(self, name, personality, domain=None, memory_window=5):
        self.name = name
        self.personality = personality
        self.domain = domain
        self.memory_window = memory_window
        
        # 1. Initialize modern Azure LLM
        # Ensure 'deployment_name' matches your Azure portal
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version="2024-02-15-preview",
            deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-35-turbo"),
            temperature=0.7
        )
        
        # 2. Storage for session history
        self.store = {}
        self._setup_conversation()

    def _get_session_history(self, session_id: str):
        if session_id not in self.store:
            self.store[session_id] = ChatMessageHistory()
        
        # Implement Window Memory (k=memory_window)
        # 1 interaction = 2 messages (Human + AI)
        max_messages = self.memory_window * 2
        if len(self.store[session_id].messages) > max_messages:
            self.store[session_id].messages = self.store[session_id].messages[-max_messages:]
        return self.store[session_id]

    def _setup_conversation(self):
        # 3. Build System Prompt
        system_msg = f"You are {self.name}. {self.personality}"
        if self.domain:
            system_msg += f"\n\nYou specialize in {self.domain}."
        system_msg += "\n\nProvide helpful, clear, and friendly responses."

        # 4. Modern ChatPrompt with Placeholder for history
        prompt = ChatPromptTemplate.from_messages([
            ("system", system_msg),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{input}"),
        ])

        # 5. Create Chain (LCEL) and wrap with history
        chain = prompt | self.llm
        self.conversation = RunnableWithMessageHistory(
            chain,
            self._get_session_history,
            input_messages_key="input",
            history_messages_key="history",
        )

    def chat(self, user_input, session_id="default"):
        try:
            # Use .invoke() instead of .predict()
            config = {"configurable": {"session_id": session_id}}
            response = self.conversation.invoke({"input": user_input}, config)
            return response.content
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"

    def reset_memory(self, session_id="default"):
        if session_id in self.store:
            self.store[session_id].clear()
            print(f"{self.name}'s memory for {session_id} has been cleared!")

# --- Examples ---
print("=== Example 1: Study Buddy ===")
study_buddy = AICompanion(
    name="StudyBot",
    personality="You are patient and encouraging.",
    domain="mathematics and science"
)
print(f"StudyBot: {study_buddy.chat('Explain Newton\'s first law')}\n")

print("=== Example 2: Wellness Coach ===")
wellness_coach = AICompanion(
    name="WellnessBot",
    personality="You are supportive and motivating.",
    domain="health and wellness"
)
print(f"WellnessBot: {wellness_coach.chat('Give me tips for better sleep')}")

=== Example 1: Study Buddy ===
StudyBot: Sorry, I encountered an error: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}

=== Example 2: Wellness Coach ===
WellnessBot: Sorry, I encountered an error: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}


## 3.3 Interactive Console Chatbot

In [17]:
def run_interactive_chatbot():
    """
    Run an interactive chat session in the console.
    """
    print("="*60)
    print("Interactive AI Companion")
    print("Commands: 'quit' to exit, 'reset' to clear memory")
    print("="*60 + "\n")
    
    # Create chatbot
    bot = AICompanion(
        name="Companion",
        personality="You are friendly, helpful, and conversational."
    )
    
    print("Companion: Hi! I'm your AI companion. How can I help you today?\n")
    
    while True:
        # Get user input
        user_input = input("You: ").strip()
        
        # Check for commands
        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("\nCompanion: Goodbye! Have a great day!")
            break
        
        if user_input.lower() == 'reset':
            bot.reset_memory()
            continue
        
        if not user_input:
            continue
        
        # Get and display response
        response = bot.chat(user_input)
        print(f"\nCompanion: {response}\n")

# Run the interactive chatbot
# Uncomment the line below to start chatting!
# run_interactive_chatbot()

# 4. Advanced: Agents with Tools

## 4.1 Understanding Agents

**Agents** are LLMs that can:
- Use external tools
- Make decisions about which tool to use
- Chain multiple tool calls together

Think of agents as "AI that can DO things" beyond just chatting!

## 4.2 Creating Custom Tools

In [30]:
class SmartChatbot:
    def __init__(self, name, tools_list):
        self.name = name
        self.tools = tools_list
        
        # 1. Initialize modern LLM
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version="2024-02-15-preview",
            azure_deployment="gpt-35-turbo", 
            temperature=0.7
        )
        
        # 2. Storage for session history (Replacing Legacy Memory)
        self.store = {}

        # 3. Define the new Agent (built on LangGraph)
        self.agent = create_agent(
            model=self.llm,
            tools=self.tools,
            system_prompt=f"You are {self.name}, an advanced robotics assistant."
        )

    def _get_history(self, session_id: str):
        if session_id not in self.store:
            self.store[session_id] = ChatMessageHistory()
        return self.store[session_id]

    def chat(self, user_input, session_id="default"):
        try:
            # v1.0 uses .invoke() with a list of messages
            runnable = RunnableWithMessageHistory(
                self.agent,
                self._get_history,
                input_messages_key="messages", # Standard for v1.0 agents
                history_messages_key="history",
            )
            config = {"configurable": {"session_id": session_id}}
            result = runnable.invoke({"messages": [("user", user_input)]}, config)
            
            # Returns the content of the final AI message
            return result['messages'][-1].content
        except Exception as e:
            return f"Error: {str(e)}"

## 4.3 Chatbot with Tools and Memory

In [31]:
import os
import langchainhub as hub
from langchain_openai import AzureChatOpenAI
from langchain_core.tools import Tool
# New v1.0 Agent abstraction
from langchain.agents import create_agent
# Persistence replaces legacy ConversationBufferMemory
from langgraph.checkpoint.memory import MemorySaver

class SmartChatbot:
    """
    v1.0 Chatbot using create_agent and LangGraph persistence.
    """
    
    def __init__(self, name, tools_list):
        self.name = name
        self.tools = tools_list
        
        # 1. Initialize modern Azure LLM
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version="2024-02-15-preview",
            azure_deployment="gpt-35-turbo", # Use actual deployment name
            temperature=0.7
        )
        
        # 2. Setup checkpointer for memory
        # MemorySaver acts as the modern short-term 'thread' memory
        self.checkpointer = MemorySaver()
        
        # 3. Create the v1.0 Agent
        # create_agent replaces AgentExecutor + create_react_agent
        self.agent = create_agent(
            model=self.llm,
            tools=self.tools,
            system_prompt=f"You are {self.name}, an advanced robotics assistant.",
            checkpointer=self.checkpointer
        )

    def chat(self, user_input, thread_id="default_thread"):
        """
        Chat using the new .invoke() interface.
        """
        try:
            # v1.0 standard input is a list of messages
            # 'thread_id' determines which conversation history to retrieve
            config = {"configurable": {"thread_id": thread_id}}
            result = self.agent.invoke(
                {"messages": [("user", user_input)]}, 
                config
            )
            # The last message in the list is the AI's final response
            return result['messages'][-1].content
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"

    def list_tools(self):
        return [tool.name for tool in self.tools]

# Usage
smart_bot = SmartChatbot(name="SmartBot", tools_list=tools)

# 5. Production-Ready Chatbot Framework

In [33]:
import os
import json
from datetime import datetime
import langchainhub as hub
from langchain_openai import AzureChatOpenAI
from langchain_core.tools import Tool
from langchain_core.messages import HumanMessage
# Modern v1.0 imports
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver

class ProductionChatbot:
    """
    A v1.0-ready chatbot framework for Robotics/Physical AI.
    """
    
    def __init__(self, config):
        self.config = config
        self.name = config.get('name', 'Assistant')
        self.personality = config.get('personality', 'You are a helpful AI assistant.')
        self.domain = config.get('domain')
        self.memory_window = config.get('memory_window', 5)
        self.temperature = config.get('temperature', 0.7)
        self.tools = config.get('tools', [])
        
        self.conversation_log = []
        
        # 1. Initialize modern Azure LLM
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version="2024-02-15-preview",
            azure_deployment="gpt-35-turbo", # Match your Azure Portal
            temperature=self.temperature
        )
        
        # 2. Setup Persistence (Replaces ConversationBufferWindowMemory)
        self.checkpointer = MemorySaver()
        
        # 3. Setup the Agent (Handles both chain and tool modes)
        system_prompt = f"You are {self.name}. {self.personality}"
        if self.domain:
            system_prompt += f" You specialize in {self.domain}."
            
        self.agent = create_agent(
            model=self.llm,
            tools=self.tools,
            system_prompt=system_prompt,
            checkpointer=self.checkpointer
        )
    
    def chat(self, user_input, thread_id="robot_session_1"):
        timestamp = datetime.now().isoformat()
        config = {"configurable": {"thread_id": thread_id}}
        
        try:
            # v1.0 uses .invoke() with a message list
            result = self.agent.invoke(
                {"messages": [("user", user_input)]}, 
                config
            )
            response = result['messages'][-1].content
            
            self._log_interaction(timestamp, user_input, response, True)
            return {'response': response, 'timestamp': timestamp, 'success': True}
        
        except Exception as e:
            error_msg = f"Error: {str(e)}"
            self._log_interaction(timestamp, user_input, error_msg, False, str(e))
            return {
                'response': "Sorry, I encountered an error processing your request.",
                'timestamp': timestamp,
                'success': False,
                'error': str(e)
            }

    def _log_interaction(self, ts, user, assistant, success, error=None):
        log_entry = {
            'timestamp': ts, 'user': user, 'assistant': assistant, 'success': success
        }
        if error: log_entry['error'] = error
        self.conversation_log.append(log_entry)

    def reset(self, thread_id="robot_session_1"):
        """Resets specific thread memory"""
        # Note: In MemorySaver, you typically just start a new thread_id
        self.conversation_log.append({
            'timestamp': datetime.now().isoformat(),
            'action': f'reset_request_for_{thread_id}'
        })

    def get_stats(self):
        total = len([log for log in self.conversation_log if 'user' in log])
        successful = len([log for log in self.conversation_log if log.get('success')])
        return {
            'total_exchanges': total,
            'successful': successful,
            'success_rate': f"{(successful/total*100):.1f}%" if total > 0 else "0%"
        }

# 6. Best Practices & Tips

##  Key Best Practices

### 1. **Memory Management**
```python
#  Good: Use window memory to control costs
memory = ConversationBufferWindowMemory(k=5)

#  Avoid: Unlimited buffer for long conversations
# memory = ConversationBufferMemory()  # Can get expensive!
```

### 2. **Temperature Selection**
- **0.0-0.3**: Factual tasks, consistent outputs
- **0.7**: Balanced (recommended for most cases)
- **0.9-1.0**: Creative writing, varied responses

### 3. **Error Handling**
```python
try:
    response = chatbot.chat(user_input)
except Exception as e:
    print(f"Error: {e}")
    response = "Sorry, something went wrong."
```

### 4. **Prompt Engineering**
- Be specific and clear
- Provide context and examples
- Define personality and constraints
- Keep system prompts focused

### 5. **API Cost Optimization**
- Limit conversation history
- Use appropriate temperature
- Cache common responses
- Monitor token usage

### 6. **Security**
- Never commit API keys
- Use environment variables
- Validate user inputs
- Implement rate limiting

### 7. **Testing**
```python
test_cases = [
    "Hello, how are you?",
    "What can you help me with?",
    "Tell me about yourself"
]

for test in test_cases:
    response = chatbot.chat(test)
    print(f"Input: {test}")
    print(f"Output: {response}\n")
```

### 8. **Code Organization**
```
project/
├── chatbot.py          # Main chatbot class
├── tools.py            # Custom tool definitions
├── config.py           # Configuration
├── .env                # API keys (gitignored!)
```

### 9. **Common Pitfalls to Avoid**
- Storing sensitive data in prompts
- Not handling API failures
- Unlimited conversation history
- Overly complex system prompts
- Not testing edge cases

### 10. **Deployment Checklist**
- [ ] Environment variables configured
- [ ] Error handling implemented
- [ ] Memory limits set
- [ ] Logging enabled
- [ ] Rate limiting considered
- [ ] API keys secured
- [ ] Testing completed

# 7. Quick Reference Cheat Sheet

## Essential Code Snippets

### Initialize LLM
```python
from langchain_openai import AzureChatOpenAI
import os

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    deployment_name="gpt-35-turbo",
    temperature=0.7
)
```

### Create Memory
```python
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=5)
```

### Simple Conversation
```python
from langchain.chains import ConversationChain

conversation = ConversationChain(llm=llm, memory=memory)
response = conversation.predict(input="Hello!")
```

### Custom Prompt
```python
from langchain.prompts import PromptTemplate

template = "You are {role}. {input}"
prompt = PromptTemplate(
    input_variables=["role", "input"],
    template=template
)
```

### Create Tool
```python
from langchain.agents import Tool

tool = Tool(
    name="ToolName",
    func=your_function,
    description="What the tool does"
)
```

### Error Handling
```python
try:
    response = chatbot.chat(user_input)
except Exception as e:
    response = f"Error: {e}"
```